# Welcome from Women on Snowflake!

**Setting the Scene: Data Disparity in Health**  
Data disparity in health is a major challenge, and we're taking a data-driven approach to solve it. With a focus on conditions like endometriosis, what better place than Snowflake, The Data Cloud, to drive change?  
Explore more: ❄️ [Data Disparity Overview](https://www.snowflake.com/en/company/overview/end-data-disparity/)

---

## Speakers

| **Speaker**             | **LinkedIn QR Code** |
| ----------------------- | --------------------- |
|❄️ [Anastasiia Stefanska](https://www.linkedin.com/in/anastasiastefanska/)   | ![Anastasiia LinkedIn QR](https://api.qrserver.com/v1/create-qr-code/?data=https://www.linkedin.com/in/anastasiastefanska/&size=100x100) |
|❄️ [Isabella Renzetti](https://www.linkedin.com/search/results/all/?fetchDeterministicClustersOnly=true&heroEntityKey=urn%3Ali%3Afsd_profile%3AACoAABFxIHABAocOnzckmeYMs4s2-7vpyxPhO1w&keywords=isabella%20renzetti&origin=RICH_QUERY_SUGGESTION&position=0&searchId=66e25045-f97f-4a21-bb24-d6bf632ce236&sid=06o&spellCorrectionEnabled=false)       | ![Isabella LinkedIn QR](https://api.qrserver.com/v1/create-qr-code/?data=https://www.linkedin.com/in/isabellarenzetti/&size=100x100) |
|❄️ [Daria Maslovets]( https://www.linkedin.com/in/daria-maslovets/)         | ![Daria LinkedIn QR](https://api.qrserver.com/v1/create-qr-code/?data=https://www.linkedin.com/in/daria-maslovets&size=100x100) |








*Scan the QR code to connect on LinkedIn.*

---

## Women on Snowflake Group

Connect with us on LinkedIn:  
![Women on Snowflake LinkedIn QR](https://api.qrserver.com/v1/create-qr-code/?data=https://www.linkedin.com/company/women-on-snowflake&size=100x100)  
❄️ [Women on Snowflake LinkedIn Page](https://www.linkedin.com/company/women-on-snowflake)

---


## Agenda for the Hands-On Lab

1. **Setting Up the Environment** 🎯  
   Create databases, schemas, and set the working context.

2. **Quick Cortex Queries** 🎯  
   Ask Snowflake a quick question using Cortex and get instant insights.

3. **Processing Reports with Cortex** 🎯  
   Parse and summarize lengthy documents with OCR.

4. **Cortex Search & Streamlit Integration** 🎯  
   Explore study data and view search results in a live app.

5. **ML Classification with Snowflake Native ML** 🎯  
   Build and evaluate a classification model on endometriosis data.

6. **Interactive Q&A and Wrap-Up** 🎯  
   Discuss findings, share experiences, and answer questions.

---

## What You'll Learn:


 ✅ **Set up your Snowflake environment** – Create databases, schemas, and warehouses.  
 ✅ **Use Snowflake Cortex** – Analyze unstructured data and process reports with AI-powered search.  
 ✅ **Build interactive dashboards with Streamlit** – Explore study data in real-time.  
 ✅ **Run SQL and Machine Learning models** – Query, transform, and classify real-world datasets.  

By the end, you’ll be equipped to process reports, apply ML models, and build interactive apps—all within Snowflake!

---

## Data You'll Use

Stay updated with our resource pages:  
- **Data Disparity in Health**  
 ❄️ [Visit Page](https://www.snowflake.com/en/company/overview/end-data-disparity/)  
  ![Data Disparity QR](https://api.qrserver.com/v1/create-qr-code/?data=https://www.snowflake.com/en/company/overview/end-data-disparity/&size=100x100)  
  *(Date: 2025-02-25)*

- **Endometriosis Resource**  
 ❄️ [Visit Page](https://www.who.int/news-room/fact-sheets/detail/endometriosis)  
  ![Endometriosis QR](https://api.qrserver.com/v1/create-qr-code/?data=https://www.who.int/news-room/fact-sheets/detail/endometriosis&size=100x100)  
  *(Date: 2025-02-25)*


Let's go and use data and Snowflake Data Cloud to drive change in health!



---
---


## ❄️ source link | 🎯 main point | ℹ️ information | 🔔 **explanation** | ⏰ pause

---

### ℹ️ What is Snowflake Cortex?
Snowflake Cortex allows to analyze unstructured data and build generative AI applications using fully managed LLMs, RAG and text-to-SQL services. It enables multiple users to use AI services with no-code, SQL and REST API interfaces.

❄️ Explore more: [Cortex](https://docs.snowflake.com/en/guides-overview-ai-features)

### ℹ️ What is Streamlit?
Streamlit is an open-source Python framework for data scientists and AI/ML engineers to deliver interactive data apps.

❄️ Explore more: [Streamlit]( https://docs.snowflake.com/en/developer-guide/streamlit/about-streamlit)

### ℹ️ What is Cortex Search?
Cortex Search enables low-latency, high-quality “fuzzy” search over your Snowflake data. Cortex Search powers a broad array of search experiences for Snowflake users including Retrieval Augmented Generation (RAG) applications leveraging Large Language Models (LLMs).

❄️ Explore more: [Cortex Search]( https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview)

## 🎯 1. Set Up the Snowflake Environment  

1️⃣ **Sign up** for a **Snowflake free 30-day trial**: ❄️ [Snowflake Demo Account](https://signup.snowflake.com/)  

   ![Snowflake Signup QR Code](https://api.qrserver.com/v1/create-qr-code/?data=https://signup.snowflake.com/&size=100x100)

   - Select **Enterprise edition** (recommended)  
   - Choose **AWS US West Oregon**   


2️⃣ **Activate & Log in**  
   - Check your **email** for the activation link and login URL  
   - Sign in using your **registered username and password**   
   

You're now ready to explore Snowflake! 🚀  



In [ ]:
-- Creates a new database for organizing event-related data. 
-- CREATE DATABASE WID_EVENT; 

-- Creates a schema inside the database to store and organize your data.
-- CREATE SCHEMA WID_DEMO;

-- Set the current database to WID_EVENT
 USE DATABASE WID_EVENT;

-- Sets the working schema to WID_DEMO for querying and data management.
 USE SCHEMA WID_DEMO; 


Now you are ready to import the notebook from Git: use database and schema you have created!

# ❄️[Git_WiD_Notebook_Flagship_March_2025](https://tinyurl.com/8zxemde8)

# https://tinyurl.com/8zxemde8


⏰ 5 Minutes Pause - time to import Notebook!

## 🎯 2: Query Cortex for a Quick Answer

Imagine you're a researcher or a healthcare professional looking for quick insights on endometriosis. Instead of manually searching through multiple sources, what if you could just ask Snowflake and get an instant answer?

That's where Snowflake Cortex comes in! 🚀

Let’s start by asking Cortex a simple question:

🧠 **"What is endometriosis?"**

With a single SQL query, Cortex processes the request using AI-powered text generation and provides a concise, AI-generated response.
###
### ℹ️ How It Works:
This query taps into **Llama3 AI** via **Cortex**, generating a quick summary **without pulling data from external sources**.

In [ ]:
-- It asks the Cortex model to generate a one-sentence answer to the question about endometriosis.
--The answer is AI-generated and not directly pulled from external sources
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'llama3.2-3b',
    'What is endometriosis? Give 1 sentence answer'
    ) AS answer;

In [ ]:
# Display the result
_2_Cortex_Generate_Short_Answer.to_df().first()

**ℹ️ Quick Summary with Cortex**

Think of Cortex as your **AI-powered assistant** in Snowflake.

Instead of spending hours reading reports, you can **automate summarization** and focus on what truly matters.
#####
**What’s Next?**  
Now that you’ve seen how Cortex answers questions instantly, let’s explore how it can process and summarize entire documents!

⏰ 2 Minutes Pause - time to ask Cortex a question!

## 🎯 3: Processing a Report with Cortex 

ℹ️ Imagine this scenario: You’re working with a lengthy medical report on endometriosis. Instead of manually scanning through pages of text, you need a fast and efficient way to extract and summarize key information.

This is where Snowflake Cortex comes in!    With AI-powered document processing, we can:  
✔ Extract text from a PDF using OCR (Optical Character Recognition)  
✔ Store the extracted content for easy access  
✔ Summarize the document to focus on the most important insights  

Let’s walk through the process step by step.
###

**Adding a file for analysis**

❄️ [World Health Organization - Endometriosis](https://www.who.int/news-room/fact-sheets/detail/endometriosis)

###

### **Step 1: Create a Stage for the Report**  
Before we can analyze the document, we need to store it in Snowflake.

In [ ]:
-- 
CREATE or ALTER stage WID_REPORT
    ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE');


-- Add a file you want to analyze
-- we are using WHO file  from this page: https://www.who.int/news-room/fact-sheets/detail/endometriosis
LIST @WID_REPORT;

⏰ 5 Minutes Pause - time to load file EndometriosisWHO.pdf to stage!

**🔔 What’s happening here?**  
We create a storage stage (WID_REPORT) to hold the file.
Encryption ensures data security.  
The LIST command checks if the file is available for processing.  

❄️ Explore more: [Tutorial: Create a document processing pipeline with Document AI](https://docs.snowflake.com/en/user-guide/snowflake-cortex/document-ai/tutorials/create-processing-pipelines#introduction)

Now, the report is ready to be analyzed!

###
### **Step 2: Extract Text from the PDF**
Now that the report is in Snowflake, let’s use Cortex OCR to convert the document into readable text.

In [ ]:
--https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex
SELECT (
    SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
        '@WID_EVENT.WID_DEMO.WID_REPORT',
        'EndometriosisWHO.pdf',
        {'mode': 'OCR'})
    ) AS OCR;

**🔔 What’s happening here?**  

Cortex OCR scans the WHO endometriosis report and extracts all text.
This allows us to query, search, and analyze the document like any other dataset.  

Why is this important?  
Instead of manually reading the entire report, we can now work with its content as structured data!

❄️ Explore more: [PARSE_DOCUMENT](https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex)

### **Step 3: Store the Parse Result**

We now store the extracted text in a table for easy access and further processing.

Why store the result?

- Keeps the text structured and accessible for future queries.
- Makes it easy to retrieve and process the extracted content.

In [ ]:
CREATE OR REPLACE TABLE PARSE_RESULT AS
SELECT * FROM {{_3c_Parse_PDF_Report}};

### **Step 4: Summarize the Parsed Content**

Instead of reading the full text, let’s use Cortex AI to summarize it into key points.

**🔔 What’s happening here?**

Cortex analyzes the extracted text and generates a concise summary.
This helps us focus on key insights rather than reading the entire document.

Why is this useful?
- Quickly grasp key findings without scanning through pages.
- Useful for researchers, analysts, and decision-makers who need quick insights.


Now, we have a concise summary of the report—generated in just a few steps!


In [ ]:
SELECT SNOWFLAKE.CORTEX.SUMMARIZE(
    SELECT SUBSTRING(TO_VARCHAR(OCR:content), 0, 32768) FROM PARSE_RESULT
);

In [ ]:
_3g_Summarize_Parsed_Content_SQL.to_df().first()

⏰ 2 Minutes Pause - time to catch up!

## 🎯 4: Cortex Search and Streamlit Integration 
Before diving into AI-powered search and visualization, we first need to prepare and explore the data. By leveraging Snowflake Cortex and Streamlit, we can analyze real-world research on endometriosis to better understand the current state of medical studies.

The following link contains different Endometrios studies:  
❄️ [National Library of Medicine - Endometriosis studies](https://clinicaltrials.gov/search?cond=Endometriosis )

Let’s walk through the process step by step.

### **Step 1: Preparing Study Data**
To begin, we need to set up a structured table where we will store research data.

Why is this important?

- The table will store summaries of studies, their primary focus, and key outcome measures.
- This will allow us to search, filter, and analyze the research data more effectively.  

Next, we create a secure stage to store the raw study data before importing it into Snowflake.

Finally, we define a file format to correctly interpret the CSV study data.

In [ ]:
-- Create a Table for Studies
CREATE OR REPLACE TABLE ENDO_STUDIES(
  SUMMARY VARCHAR,
  STUDY VARCHAR, 
  PRIMARY_OUTCOME_MEASURES VARCHAR,
  SECONDARY_OUTCOME_MEASURES VARCHAR
);

-- Set Up a Stage for Studies Data
CREATE OR ALTER STAGE WID_STUDIES
  ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

-- Create a CSV File Format
CREATE OR REPLACE FILE FORMAT my_csv 
  TYPE = CSV, 
  SKIP_HEADER = 1,
  FIELD_OPTIONALLY_ENCLOSED_BY = '"';

LIST @WID_STUDIES;

⏰ 2 Minutes Pause - time to load file WID_Endo_studies_list_for_Cortex.csv to stage!

Our database and storage stage are ready for the research data!  

Now that we’ve set up the structure, let’s import the study data from a CSV file.
- The data now becomes queryable inside Snowflake.
- We can apply AI-powered search and analysis on structured research data.

In [ ]:
-- Import the Data
COPY INTO ENDO_STUDIES FROM
(SELECT $6, $2, $10, $11
 FROM '@wid_studies/WID_Endo_studies_list_for_Cortex.csv'
 (FILE_FORMAT => my_csv)
)
FORCE = TRUE;

In [ ]:
-- Let’s verify that our imported research data is correctly stored in Snowflake.
SELECT * FROM ENDO_STUDIES;

🔜 **What’s Next?**
Now that our data is ready, we can:
- Use Cortex Search to quickly find relevant studies.
- Build an interactive research dashboard using Streamlit.
- Gain insights into trends and patterns in endometriosis research.

###
### **Step 2: Creating and Querying the Cortex Search Service**

Now that we have research data loaded, the next step is making it **searchable and easy to explore**.  
Instead of manually filtering through study summaries, we can use **Cortex Search** to find relevant studies instantly.  
Let’s **set up a search service**, index the data, and run a **query** to find studies related to **prevention**.

######
Before we can search, we need to create a search service that indexes our study data:



In [ ]:
-- Create the Search Service
CREATE OR REPLACE CORTEX SEARCH SERVICE study_search_service
  ON study
  ATTRIBUTES summary
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-m-v1.5'
  AS (
    SELECT
        summary,
        study,
        primary_outcome_measures
    FROM ENDO_STUDIES
);

🔔 **What’s happening here?**

- We create a search service that lets us query the ENDO_STUDIES table.
- It indexes the study summaries, titles, and outcomes for faster retrieval.
- The EMBEDDING_MODEL improves search relevance by understanding context.

Now, we have an AI-powered search engine for our research data!

######
Our search service is ready, let’s now run a query to find studies about prevention:

In [ ]:
-- Query the Search Service
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
      'wid_event.wid_demo.study_search_service',
      '{
        "query": "prevention",
        "columns":[
            "summary",
            "study"
        ]
      }'
  )
)['results'] as results;

🔔 **What’s happening here?**

- We search for studies related to "prevention" using Cortex Search.
- The query looks for matches in the summary and study columns.
- The results are returned as a structured JSON format for easy interpretation.

Now, we can instantly find relevant research studies based on keywords!


🔜 **What’s Next?**  
Now that we can search studies with AI, we can:
- Refine search queries to find specific research topics.
- Use Streamlit to create an interactive search app for better visualization.
- Analyze key patterns in endometriosis research.

###
### **Step 3: Streamlit Code for Cortex AI Search**

Now that we have **Cortex Search set up**, let’s make it **more interactive**!   
Instead of manually writing queries, we’ll build a Streamlit app that allows users to search the study data using a simple interface.

With just a few clicks, we will be able to:
- Enter a **search query**
- Define **search limits**
- View **instant results from Cortex AI**


In [ ]:
# Import python packages
import streamlit as st
from snowflake.core import Root
from snowflake.snowpark.context import get_active_session

# Constants
DB = "WID_EVENT"
SCHEMA = "WID_DEMO"
SERVICE = "study_search_service"
BASE_TABLE = "WID_EVENT.WID_DEMO.ENDO_STUDIES"
#ARRAY_ATTRIBUTES = {"SUMMARY"}


def get_column_specification():
    """
    Returns the name of the search column and a list of the names of the attribute columns
    for the provided cortex search service
    """
    session = get_active_session()
    search_service_result = session.sql(f"DESC CORTEX SEARCH SERVICE {DB}.{SCHEMA}.{SERVICE}").collect()[0]
    st.session_state.attribute_columns = search_service_result.attribute_columns.split(",")
    st.session_state.search_column = search_service_result.search_column
    st.session_state.columns = search_service_result.columns.split(",")

def init_layout():
    st.title("Cortex AI Search")
    st.markdown(f"Querying service: `{DB}.{SCHEMA}.{SERVICE}`".replace('"', ''))

def query_cortex_search_service(query):
    """
    Queries the cortex search service in the session state and returns a list of results
    """
    session = get_active_session()
    cortex_search_service = (
        Root(session)
        .databases[DB]
        .schemas[SCHEMA]
        .cortex_search_services[SERVICE]
    )
    context_documents = cortex_search_service.search(
        query,
        columns=st.session_state.columns,
        limit=st.session_state.limit)
    return context_documents.results



def init_search_input():
    st.session_state.query = st.text_input("Query")

def init_limit_input():
    st.session_state.limit = st.number_input("Limit", min_value=1, value=5)


def display_search_results(results):
    """
    Display the search results in the UI
    """
    st.subheader("Search results")
    for i, result in enumerate(results):
        result = dict(result)
        container = st.expander(f"[Result {i+1}]", expanded=True)

        # Add the result text.
        container.markdown(result[st.session_state.search_column])

        # Add the attributes.
        for column, column_value in sorted(result.items()):
            if column == st.session_state.search_column:
                continue
            container.markdown(f"**{column}**: {column_value}")


def main():
    init_layout()
    get_column_specification()
    init_limit_input()
    init_search_input()

    if not st.session_state.query:
        return
    results = query_cortex_search_service(
        st.session_state.query
    )
    display_search_results(results)


if __name__ == "__main__":
    #st.set_page_config(page_title="Cortex AI Search and Summary", layout="wide")
    main()



🔔 **What's happening here?**

This script creates a simple Streamlit app for searching study data using Snowflake Cortex Search.

How It Works:

- Retrieves **searchable columns** from the search service.
- Displays a **text box** for user input.
- Runs the **Cortex AI search query** based on the input.
- Shows the **results interactively** in the app.

Now, users can explore medical research data without writing SQL!

⏰ 5 Minutes Pause - time to refresh!

## 🎯 5. ML Classification with Snowflake Native ML 

Endometriosis is often difficult to diagnose due to the wide range of symptoms and delayed medical recognition.  
What if we could use Machine Learning (ML) to help identify patients at risk based on their symptoms?

With Snowflake Native ML, we can:
- Prepare structured patient data.
- Train an ML model to classify diagnosis possibilities.
- Use Snowflake’s ML capabilities without external tools.

Let’s prepare our dataset to begin this journey.

######
❄️ [Endometriosis Dataset](https://www.kaggle.com/datasets/michaelanietie/endometriosis-dataset)

![Kaggle Endometriosis Dataset QR Code](https://api.qrserver.com/v1/create-qr-code/?data=https://www.kaggle.com/datasets/michaelanietie/endometriosis-dataset&size=100x100)


######
**Step 1: Preparing the Raw Data**  
Before we can apply ML, we need to organize the data in Snowflake: we first define a stage to store incoming patient records:





In [ ]:
-- Create a Stage for Raw Data
CREATE OR ALTER STAGE WID_RAW_DATA;

-- Let's make sure the data is available before processing
LIST @WID_RAW_DATA;

⏰ 2 Minutes Pause - time to load file structured_endometriosis_data.csv to stage!

Each record needs a **unique identifier** for tracking. We create a **sequence generator** to automate this.

Then, we define a **structured table** to store patient health metrics and diagnosis labels:

In [ ]:
-- Creating a Unique ID for Each Record
CREATE OR REPLACE SEQUENCE seq_01 START = 1 INCREMENT = 1;

-- Create the Data Table
CREATE OR REPLACE TABLE ENDO_DATA (
  ROW_ID NUMBER DEFAULT seq_01.NEXTVAL,
  AGE NUMBER,
  MENSTRUAL_IRREGULARITY BOOLEAN,
  CHRONIC_PAIN_LEVEL NUMBER(38,5),
  HORMON_LEVEL_ABNORMALITY BOOLEAN,
  INFERTILITY BOOLEAN,
  BMI NUMBER(38,5),
  DIAGNOSIS BOOLEAN
);

In [ ]:
SELECT * from ENDO_DATA;

🔔 **What’s in this table?**

- **Age** → Captures patient age.
- **Menstrual Irregularity** → Tracks cycle irregularities.
- **Chronic Pain Level** → Quantifies pain severity.
- **Hormone Level Abnormality** → Flags hormonal imbalances.
- **Infertility** → Indicates potential fertility issues.
- **BMI** → Includes body mass index data.
- **Diagnosis** → ML will predict this column (1 = endometriosis, 0 = no diagnosis).

Now, we have a structured dataset ready for ML training!

###
🔜 **What’s Next?**
Now that we have **patient data organized**, we can:
- Load real-world patient data into the table.
- Train an ML model using **Snowflake’s Native ML capabilities**.
- Predict **endometriosis diagnosis** based on symptoms.



In [ ]:
-- Load Data into the Table
COPY INTO ENDO_DATA (
    AGE,
    MENSTRUAL_IRREGULARITY,
    CHRONIC_PAIN_LEVEL,
    HORMON_LEVEL_ABNORMALITY,
    INFERTILITY,
    BMI,
    DIAGNOSIS
)
FROM @WID_RAW_DATA
FILE_FORMAT = (TYPE = CSV SKIP_HEADER = 1);

In [ ]:
SELECT DIAGNOSIS, COUNT(*) FROM ENDO_DATA GROUP BY 1;

**Step 2: Splitting Data into Training and Test Sets**

Now that we have a structured dataset, it's time to prepare the data for machine learning. A key step in building an effective ML model is splitting the data into:

- **Training Data (80%)** – Used to teach the model patterns.  
- **Test Data (20%)** – Used to evaluate how well the model generalizes.

By separating the data, we ensure that our ML model learns effectively while also being tested on unseen data to check its accuracy.

We start by randomly selecting **80% of the records** for training:



In [ ]:
--Create Training Set = 80% of data
CREATE OR REPLACE TABLE ENDO_DATA_TRAINING AS
SELECT * FROM ENDO_DATA SAMPLE (80);

SELECT * FROM ENDO_DATA_TRAINING;

🔔 What is happening here?

- The SAMPLE (80) function randomly selects 80% of the data.
- We store this selection in ENDO_DATA_TRAINING, which will be used to train the model.
- The SELECT statement confirms that the training set was created successfully.

Now, we have a training dataset ready for model learning!

####
The remaining **20% of the data** is set aside for testing. We ensure that no row from the training set appears in the test set:

In [ ]:
--Create Test Set
CREATE OR REPLACE TABLE ENDO_DATA_TEST AS
SELECT * FROM ENDO_DATA
WHERE ROW_ID NOT IN (
    SELECT ROW_ID FROM ENDO_DATA_TRAINING
);

SELECT * FROM ENDO_DATA_TEST;

🔔 **What’s happening here?**

- We exclude records already in the training set.
- This ensures that our test data is completely separate from the training data.
- The test set will be used to evaluate the ML model’s performance.

Now, we have a separate test dataset for evaluation! 






**Step 3: Build and Evaluate the Classification Model**

Now that our **data is prepared and split into training and test sets**, it's time to **train the ML model**!

We will use **Snowflake Native ML** to build a **binary classification model** that predicts **whether a patient is likely to be diagnosed with endometriosis** based on their symptoms.
###

Before training the model, we create a view that selects only the relevant features for classification.

In [ ]:
-- Create a View for Binary Classification
CREATE OR REPLACE VIEW binary_classification_view AS
SELECT AGE,
       MENSTRUAL_IRREGULARITY,
       CHRONIC_PAIN_LEVEL,
       HORMON_LEVEL_ABNORMALITY,
       INFERTILITY,
       BMI,
       DIAGNOSIS
FROM ENDO_DATA_TRAINING;

-- View a Sample of the Data
SELECT * FROM binary_classification_view ORDER BY RANDOM(42) LIMIT 5;


🔔 **Why create a view?**

- It filters only the relevant columns needed for training the ML model.
- Keeps the original training dataset intact while focusing on useful features.
- Allows easy modifications if we need to add or remove features later.

Now, we have a structured dataset for training!

In [ ]:
-- Build the Classification Model
CREATE OR REPLACE SNOWFLAKE.ML.CLASSIFICATION model_binary(
    INPUT_DATA => SYSTEM$REFERENCE('view', 'binary_classification_view'),
    TARGET_COLNAME => 'DIAGNOSIS'
);

⏰ 5 Minutes Pause - time to create a model!

🔔 **What’s happening here?**

- We use Snowflake’s built-in ML capabilities to create a classification model.
- The model is trained using binary_classification_view as input.
- The target column is DIAGNOSIS (0 = No Endometriosis, 1 = Diagnosed).

Our ML model is training!

######
Now that our **model is trained**, we use it to **predict endometriosis diagnosis** on the test dataset.



In [ ]:
--Generate Predictions and Show Evaluation Metrics
CREATE OR REPLACE TABLE PREDICTION_RESULTS as
SELECT *, model_binary!PREDICT(INPUT_DATA => {*})
    AS predictions FROM ENDO_DATA_TEST;

🔔 **What’s happening here?**

- We apply our trained model (model_binary) to the test dataset (ENDO_DATA_TEST).
- The model generates predicted diagnosis values (1 = Diagnosed, 0 = Not Diagnosed).
- The results are stored in PREDICTION_RESULTS for further analysis.

In [ ]:
-- 🎯 Show evaluation metrics
CALL model_binary!SHOW_EVALUATION_METRICS();
--CALL model_binary!SHOW_GLOBAL_EVALUATION_METRICS();
--CALL model_binary!SHOW_THRESHOLD_METRICS();
--CALL model_binary!SHOW_CONFUSION_MATRIX();

🔔 **What do these metrics mean?**



**CALL model_binary!SHOW_EVALUATION_METRICS**() Displays key performance metrics like accuracy, precision, and recall.   
**SHOW_GLOBAL_EVALUATION_METRICS**() Provides overall model performance across all data.
**SHOW_THRESHOLD_METRICS**() Helps assess decision threshold impacts.
**SHOW_CONFUSION_MATRIX**() Shows true positives, false positives, and false negatives to evaluate prediction reliability.

In [ ]:
-- Check Prediction Accuracy
WITH a AS (
    SELECT *,
           TO_BOOLEAN(UPPER(AS_VARCHAR(PREDICTIONS:class))) AS prediction
    FROM PREDICTION_RESULTS
)
SELECT prediction,
       diagnosis,
       prediction = diagnosis AS correct_prediction,
       COUNT(row_id)
FROM a
GROUP BY 1, 2, 3;

In [ ]:
--select diagnosis, count(*) from ENDO_DATA_TRAINING group by 1;
CREATE OR REPLACE TABLE ENDO_DATA_TRAINING_BALANCED_DATA as
(with a as (select row_id from ENDO_DATA_TRAINING
where diagnosis = false),
b as (select distinct row_id from a sample (32)),
c as (select * from ENDO_DATA_TRAINING
where row_id not in (select * from b))
select * from c
);

CREATE OR REPLACE VIEW binary_classification_view_balanced_data AS
SELECT AGE,
       MENSTRUAL_IRREGULARITY,
       CHRONIC_PAIN_LEVEL,
       HORMON_LEVEL_ABNORMALITY,
       INFERTILITY,
       BMI,
       DIAGNOSIS
FROM ENDO_DATA_TRAINING_BALANCED_DATA;

CREATE OR REPLACE SNOWFLAKE.ML.CLASSIFICATION model_binary_balanced_data(
    INPUT_DATA => SYSTEM$REFERENCE('view', 'binary_classification_view_balanced_data'),
    TARGET_COLNAME => 'DIAGNOSIS'
);

In [ ]:
--Generate Predictions and Show Evaluation Metrics
CREATE OR REPLACE TABLE PREDICTION_RESULTS_BALANCED_DATA as
SELECT *, model_binary_balanced_data!PREDICT(INPUT_DATA => {*})
    AS predictions FROM ENDO_DATA_TEST;

In [ ]:
CALL model_binary_balanced_data!SHOW_EVALUATION_METRICS();

🔔 **What is happening here**



**CALL model_binary_balanced_data** is using balanced data with ±50% of rows identified as DIAGNOSIS = TRUE.   
New model achieves higher results in both precision and recall for the Class TRUE, which in our data corresponds to diagnosis, compared to model_binary model, where ±40% of data corresponds to same.


By increasing the share of feature representation in the training data we were able to immediately achieve higher model accuracy in predicting the feature without modifying the data.

Solving Data Disparity can save lives.

## 🎯 6. Interactive Q&A and Wrap-Up 

Through this hands-on lab, we explored how Snowflake Native ML can be used to predict endometriosis diagnosis based on patient data. Here’s a recap of what we achieved:

- What insights did you gain from the lab?
- How can this be applied to real-world challenges?
- What would you improve or explore further?

## Conclusion 🔚
**1. Setting Up:**
We created the database, schema, and set the working context.

**2. Cortex Summaries:**
Cortex helps us get short summaries from long reports.

**3. Cortex Search:**
We loaded study data, built a Cortex search service, and integrated it with Streamlit.

**4. ML Classification:** 
We prepared endometriosis data, built a classification model, and evaluated its predictions.

As a conclusion, we invite you to go deeper into the possibilities of data platforms, as you can see - even small step can help to tackle the **data disparity**.

## ❄️ Thank you! ❄️

## Speakers

| **Speaker**             | **LinkedIn QR Code** |
| ----------------------- | --------------------- |
| ❄️ [Anastasiia Stefanska](https://www.linkedin.com/in/anastasiastefanska/)     | ![Anastasiia LinkedIn QR](https://api.qrserver.com/v1/create-qr-code/?data=https://www.linkedin.com/in/anastasiastefanska/&size=100x100) |
| ❄️ [Isabella Renzetti](https://www.linkedin.com/search/results/all/?fetchDeterministicClustersOnly=true&heroEntityKey=urn%3Ali%3Afsd_profile%3AACoAABFxIHABAocOnzckmeYMs4s2-7vpyxPhO1w&keywords=isabella%20renzetti&origin=RICH_QUERY_SUGGESTION&position=0&searchId=66e25045-f97f-4a21-bb24-d6bf632ce236&sid=06o&spellCorrectionEnabled=false)        | ![Isabella LinkedIn QR](https://api.qrserver.com/v1/create-qr-code/?data=https://www.linkedin.com/in/isabellarenzetti/&size=100x100) |
| ❄️ [Daria Maslovets]( https://www.linkedin.com/in/daria-maslovets/)          | ![Daria LinkedIn QR](https://api.qrserver.com/v1/create-qr-code/?data=https://www.linkedin.com/in/daria-maslovets&size=100x100) |

*Scan the QR code to connect on LinkedIn.*



   
